In [1]:
'''Load Packages'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings
from sklearn.cluster import KMeans 
import random
from sklearn import metrics 
from scipy.spatial.distance import cdist

In [2]:
# function to transform into numerical bins by normal/gaussian distribution
def categorical_transform(col):
    classnames, factorval, rank = np.unique(col, return_inverse = True,return_counts = True)
    
      
    temp = np.sort(rank)
    temp = np.concatenate((temp[::2], temp[1::2][::-1]))
    rank = random.sample(range(1, len(temp)+1),len(temp))
    
    category_count = {}
    for i in range(len(classnames)):
        category_count[classnames[i]] = rank[i]
    
    check_list = list(col)
    for i in range(len(check_list)):
        for key, value in category_count.items():
            if check_list[i] == key:
                factorval[i] = value 
                break
    
    return category_count, factorval

In [3]:
def cat_to_names(df,references):
    ref = df.columns[df.columns.isin(references.keys())].to_list()
    temp_df = df.copy()
    for key in ref:
        vals = references[key]
        for label_key in vals.keys():
            temp_df[key] = temp_df[key].replace(vals[label_key],label_key)
    return temp_df

In [4]:
## load data set
carmax_df = pd.read_csv('CaseDataRemodeled.csv')
new_df = pd.read_csv('newdata.csv')

In [5]:
carmax_df['job_assign'] = new_df['job_assign']

In [6]:
cluster_df = carmax_df.copy().drop(['customer_age','customer_previous_purchase','customer_distance_to_dealer',
                             'post_purchase_satisfaction','vehicle_warranty_used','subsequent_purchases',
                             'purchase_price_UB','purchase_class'], axis=1)

In [7]:
## references of transformation for later 
references = {}

# transform to numerical values 
for i in cluster_df.columns:
    if cluster_df[i].dtype == 'object':
        classes = cluster_df[i]
        references[i], cluster_df[i] = categorical_transform(classes)

In [8]:
cluster_df['purchase_class'] = carmax_df['purchase_class']

In [9]:
sedan = cluster_df[cluster_df.purchase_class=='sedan'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
truck = cluster_df[cluster_df.purchase_class=='truck'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
suv = cluster_df[cluster_df.purchase_class=='suv'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
wagon = cluster_df[cluster_df.purchase_class=='wagon'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
van = cluster_df[cluster_df.purchase_class=='van'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
coupe = cluster_df[cluster_df.purchase_class=='coupe'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)
convertible = cluster_df[cluster_df.purchase_class=='convertible'].copy().reset_index(drop = True).drop('purchase_class', axis = 1)

vehicle_classes = [sedan,truck,suv,wagon,van,coupe,convertible]

### Elbow Method to determine k for each cluster of vehicle class

In [14]:
# vehicle_classes_name = ['sedan','truck','suv','wagon','van','coupe','convertible']
# index = 0

# for group in vehicle_classes:
    
#     distortions = [] 
#     inertias = []  
#     mapping2 = {} 
#     K = range(1,15) 

#     X = group.values[:,:]
#     X = np.nan_to_num(X)
#     cluster_dataset = StandardScaler().fit_transform(X)
#     cluster_dataset
    
#     for k in K: 

#         #Building and fitting the model 
#         kmeanModel = KMeans(init="k-means++", n_clusters=k, n_init=50)
#         kmeanModel.fit(cluster_dataset)     
      
#         distortions.append(sum(np.min(cdist(cluster_dataset, kmeanModel.cluster_centers_, 
#                       'euclidean'),axis=1)) / cluster_dataset.shape[0]) 
#         inertias.append(kmeanModel.inertia_) 
  
#         mapping1[k] = sum(np.min(cdist(cluster_dataset, kmeanModel.cluster_centers_, 
#                  'euclidean'),axis=1)) / cluster_dataset.shape[0] 
#         mapping2[k] = kmeanModel.inertia_ 
    
#     print(vehicle_classes_name[index])
#     index += 1
    
#     for key,val in mapping1.items(): 
#         print(str(key)+' : '+str(val)) 
        
#     plt.plot(K, distortions, 'bx-') 
#     plt.xlabel('Values of K') 
#     plt.ylabel('Distortion') 
#     plt.title('The Elbow Method using Distortion') 
#     plt.show() 
    
#     for key,val in mapping2.items(): 
#         print(str(key)+' : '+str(val)) 
    
#     plt.plot(K, inertias, 'bx-') 
#     plt.xlabel('Values of K') 
#     plt.ylabel('Inertia') 
#     plt.title('The Elbow Method using Inertia') 
#     plt.show()    

In [15]:
from sklearn.preprocessing import StandardScaler

k_values = [5,5,6,6,5,6,6]
index = 0

for group in vehicle_classes:
    X = group.values[:,:]
    X = np.nan_to_num(X)
    cluster_dataset = StandardScaler().fit_transform(X)
    cluster_dataset

    num_clusters = k_values[index]
    index += 1

    k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=50,algorithm = "elkan")
    k_means.fit(cluster_dataset)
    labels = k_means.labels_
    group["Labels"] = labels

In [16]:
sedanCluster = cluster_with_names = cat_to_names(sedan,references)
truckCluster = cluster_with_names = cat_to_names(truck,references)
suvCluster = cluster_with_names = cat_to_names(suv,references)
wagonCluster = cluster_with_names = cat_to_names(wagon,references)
vanCluster = cluster_with_names = cat_to_names(van,references)
coupeCluster = cluster_with_names = cat_to_names(coupe,references)
convertibleCluster = cluster_with_names = cat_to_names(convertible,references)

In [17]:
truckCluster

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,trade_in,vehicle_financing,customer_income,customer_gender,AgeBin,made_in,job_assign,Labels
0,FORD,F150,2007,15001.0,0,0,0 - 20000,F,Fifties,UNITED STATES,"Entertainers and Performers, Sports and Relate...",1
1,NISSAN,FRONTIER,2012,20001.0,0,1,20001 - 40000,M,Twenties,JAPAN,Bakers,4
2,FORD,F150,2006,15001.0,0,1,40001 - 60000,U,Twenties,UNITED STATES,Glaziers,2
3,TOYOTA,TUNDRA,2009,20001.0,1,1,60001 - 80000,M,Thirties,JAPAN,First-Line Supervisors of Construction Trades ...,4
4,NISSAN,FRONTIER,2009,20001.0,0,1,40001 - 60000,F,Twenties,JAPAN,Real Estate Sales Agents,4
...,...,...,...,...,...,...,...,...,...,...,...,...
28762,TOYOTA,TUNDRA,2004,15001.0,0,1,40001 - 60000,M,Thirties,JAPAN,Exercise Physiologists,4
28763,CHEVROLET,SILVERADO 1500,2011,20001.0,0,1,40001 - 60000,F,Twenties,UNITED STATES,"Carpet, Floor, and Tile Installers and Finishers",0
28764,FORD,F150,2009,20001.0,1,1,100001 - 120000,M,Forties,UNITED STATES,Sales Engineers,3
28765,TOYOTA,TACOMA,2007,15001.0,0,1,20001 - 40000,M,Twenties,JAPAN,"Cutters and Trimmers, Hand",4


In [18]:
truckCluster[truckCluster.Labels==0].groupby('purchase_model').count().sort_values(by='purchase_make', ascending = False)
#cluster_df.groupby('Labels').mean()

,purchase_make,purchase_vehicle_year,purchase_price_LB,trade_in,vehicle_financing,customer_income,customer_gender,AgeBin,made_in,job_assign,Labels
purchase_model,,,,,,,,,,,
SILVERADO 1500,3747,3747,3747,3747,3747,3747,3747,3747,3747,3747,3747
SIERRA 1500,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312
AVALANCHE 1500,578,578,578,578,578,578,578,578,578,578,578
SILVERADO 2500,70,70,70,70,70,70,70,70,70,70,70
CANYON,54,54,54,54,54,54,54,54,54,54,54
MARK LT,33,33,33,33,33,33,33,33,33,33,33
SONOMA,18,18,18,18,18,18,18,18,18,18,18
SILVERADO 1500 HD,13,13,13,13,13,13,13,13,13,13,13
L200,10,10,10,10,10,10,10,10,10,10,10


In [19]:
truck.groupby('Labels').mean()

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,trade_in,vehicle_financing,customer_income,customer_gender,AgeBin,made_in,job_assign
Labels,,,,,,,,,,,
0,34.883310,547.123544,2008.687800,18422.007539,0.425291,0.771933,4.273132,2.086532,6.119945,2.000000,520.131254
1,17.003285,204.879175,2008.813652,18184.975178,0.413944,0.000000,3.593721,2.074284,6.210805,2.384194,513.984851
2,23.655317,182.735392,2007.379887,14707.391574,0.394027,0.997114,3.827875,2.108642,6.108065,2.000144,521.935651
3,20.312773,245.136652,2010.914320,28269.544228,0.683856,0.895478,7.585085,2.056327,6.028163,2.052360,457.542443
4,12.611790,196.727505,2008.520533,18155.772769,0.491513,1.000000,4.886111,2.108779,6.116262,3.000000,502.368498


In [20]:
truckCluster.groupby('Labels').agg(lambda x: pd.Series.mode(x)[0])

,purchase_make,purchase_model,purchase_vehicle_year,purchase_price_LB,trade_in,vehicle_financing,customer_income,customer_gender,AgeBin,made_in,job_assign
Labels,,,,,,,,,,,
0,CHEVROLET,SILVERADO 1500,2011,20001.0,0,1,40001 - 60000,M,Forties,UNITED STATES,General Internal Medicine Physicians
1,FORD,F150,2010,15001.0,0,0,?,M,Forties,UNITED STATES,General Internal Medicine Physicians
2,FORD,RAM 1500,2008,15001.0,0,1,40001 - 60000,M,Forties,UNITED STATES,General Internal Medicine Physicians
3,FORD,F150,2012,25001.0,1,1,60001 - 80000,M,Forties,UNITED STATES,"Physicians, All Other; and Ophthalmologists, E..."
4,TOYOTA,TACOMA,2007,15001.0,0,1,40001 - 60000,M,Forties,JAPAN,"Physicians, All Other; and Ophthalmologists, E..."


In [21]:
sorted(references['purchase_model'].items(), key=lambda x: x[1])

[('C350', 1),
 ('MONTANA', 2),
 ('300', 3),
 ('5-Sep', 4),
 ('SUNFIRE', 5),
 ('GALANT', 6),
 ('CC', 7),
 ('ALTIMA', 8),
 ('X6 HYBRID', 9),
 ('VERSA NOTE', 10),
 ('LUCERNE', 11),
 ('300M', 12),
 ('ACCORD', 13),
 ('L300', 14),
 ('STS', 15),
 ('i-280', 16),
 ('E55 AMG', 17),
 ('S55 AMG', 18),
 ('BONNEVILLE', 19),
 ('TAHOE', 20),
 ('M5', 21),
 ('G500', 22),
 ('CLS550', 23),
 ('CL550', 24),
 ('ESCALADE HYBRID', 25),
 ('VUE HYBRID', 26),
 ('RODEO', 27),
 ('CADENZA', 28),
 ('C/V', 29),
 ('NV 2500', 30),
 ('NAVIGATOR', 31),
 ('SILVERADO 3500', 32),
 ('E320', 33),
 ('C320', 34),
 ('SLK55 AMG', 35),
 ('VERONA', 36),
 ('SUBURBAN 1500', 37),
 ('IMPALA', 38),
 ('SEDONA', 39),
 ('SL55 AMG', 40),
 ('CONTINENTAL GT', 41),
 ('XK8', 42),
 ('VENZA', 43),
 ('NAVIGATOR L', 44),
 ('C70', 45),
 ('HIGHLANDER HYBRID', 46),
 ('G25', 47),
 ('ECLIPSE', 48),
 ('E350', 49),
 ('9-2X', 50),
 ('AVALON', 51),
 ('MALIBU CLASSIC', 52),
 ('Z4', 53),
 ('MONTE CARLO', 54),
 ('OUTBACK', 55),
 ('SLK250', 56),
 ('COOPER', 57),

In [22]:
references['purchase_model']

{'128': 339,
 '135': 510,
 '1M': 279,
 '200': 525,
 '3-Sep': 162,
 '300': 3,
 '300 C SRT': 144,
 '300M': 12,
 '300ZX': 208,
 '320': 67,
 '325': 462,
 '328': 529,
 '330': 295,
 '335': 143,
 '335 HYBRID': 615,
 '350Z': 491,
 '370Z': 71,
 '4RUNNER': 422,
 '5-Sep': 4,
 '500': 240,
 '525': 548,
 '528': 239,
 '530': 534,
 '535': 220,
 '540': 590,
 '545': 178,
 '550': 377,
 '626': 205,
 '640': 369,
 '640 B640': 288,
 '645': 187,
 '650': 175,
 '740': 226,
 '745': 470,
 '750': 300,
 '750 HYBRID': 118,
 '760': 585,
 '9-2X': 50,
 '911': 451,
 'A3': 194,
 'A4': 262,
 'A5': 392,
 'A6': 530,
 'A7': 156,
 'A8': 526,
 'ACADIA': 338,
 'ACCENT': 391,
 'ACCORD': 13,
 'ACCORD CROSSTOUR': 160,
 'ACCORD HYBRID': 59,
 'AERIO': 274,
 'ALERO': 227,
 'ALLROAD': 584,
 'ALPINA B7': 304,
 'ALTIMA': 8,
 'ALTIMA HYBRID': 100,
 'AMANTI': 177,
 'ARMADA': 312,
 'ASCENDER': 591,
 'ASCENDER EXT': 92,
 'ASPEN': 428,
 'ASPEN HYBRID': 275,
 'ASTRA': 498,
 'ASTRO': 244,
 'ATS': 133,
 'AURA': 405,
 'AURA HYBRID': 374,
 'AUROR

In [23]:
export_groups = [sedanCluster,truckCluster,suvCluster,wagonCluster,vanCluster,coupeCluster,convertibleCluster]

In [24]:
cluster_names = ['sedanCluster','truckCluster','suvCluster','wagonCluster','vanCluster','coupeCluster'
                        ,'convertibleCluster']
index = 0
for group in export_groups:
    for x in group.Labels.unique():
        group[group.Labels==x].to_csv(cluster_names[index]+str(x)+'.csv', index = False)
    index += 1